## Volume

doesn't use because yaml config doesn't support relative path.

# load docker image to k8s

```
kind load docker-image ml-zoomcamp_inference-ch2:latest
kind load docker-image ml-zoomcamp_inference-gateway
```

List CRI images available for k8s.

In [3]:
!docker exec -it kind-control-plane crictl image

IMAGE                                             TAG                  IMAGE ID            SIZE
docker.io/kindest/kindnetd                        v20220510-4929dd75   31d1852b05abc       43.8MB
docker.io/kindest/local-path-helper               v20220512-507ff70b   9bc13867d19ca       2.75MB
docker.io/kindest/local-path-provisioner          v0.0.22-kind.0       7902f9a1c54fa       15.6MB
docker.io/library/ml-zoomcamp_inference-ch2       latest               1d5b92ee3172b       1GB
docker.io/library/ml-zoomcamp_inference-gateway   latest               1ff49d080c415       2.32GB
k8s.gcr.io/coredns/coredns                        v1.8.6               edaa71f2aee88       12.3MB
k8s.gcr.io/etcd                                   3.5.3-0              a9a710bb96df0       81.1MB
k8s.gcr.io/kube-apiserver                         v1.24.0              acde05a590759       75.5MB
k8s.gcr.io/kube-controller-manager                v1.24.0              b1d312727171f       63.8MB
k8s.gcr.io/kube-proxy    

# Network access

- internal
    through local dns
        - `<service_name>.<namespace>`, e.g gateway.default
        - `<service_name>.<namespace>.svc.cluster.local

- external
    - service, LoadBalancer, NodePort
        - NodePort doesn't work in mac, can't access the internal node ip.
    - port forward
        kubectl port-forward services/gateway 9999:8000
    - ingress

## Common commands

- `get`, return list
- `describe`, show detail

In [7]:
!kubectl get service -o wide

NAME                   TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE    SELECTOR
ch2-inference-server   ClusterIP   10.96.146.45    <none>        9527/TCP   158m   app=ch2-inference-server
gateway                ClusterIP   10.96.251.106   <none>        8000/TCP   132m   app=gateway
kubernetes             ClusterIP   10.96.0.1       <none>        443/TCP    2d4h   <none>


In [8]:
!kubectl get deployment -o wide

NAME                   READY   UP-TO-DATE   AVAILABLE   AGE    CONTAINERS             IMAGES                          SELECTOR
ch2-inference-server   1/1     1            1           169m   ch2-inference-server   ml-zoomcamp_inference-ch2       app=ch2-inference-server
gateway                1/1     1            1           141m   gateway                ml-zoomcamp_inference-gateway   app=gateway


In [9]:
!kubectl get pods -o wide

NAME                                    READY   STATUS    RESTARTS   AGE    IP            NODE                 NOMINATED NODE   READINESS GATES
ch2-inference-server-69d4c4f76b-g2gdj   1/1     Running   0          169m   10.244.0.8    kind-control-plane   <none>           <none>
gateway-565ffcc8bc-p2479                1/1     Running   0          117m   10.244.0.10   kind-control-plane   <none>           <none>
